In [ ]:
#importing libraries

import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Importing Decision Tree Classifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [13]:
df = pd.read_csv("heart.csv")
col_names = list(df.columns)

In [14]:
features = ['thal', 'thalach', 'slope', 'exang', 'cp', 'oldpeak', 'ca', 'sex']

In [15]:
X = df[features]
y = df.target
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 0)

In [16]:
clf = DecisionTreeClassifier(criterion = "gini",random_state = 0,max_depth = 6,max_features = 3,max_leaf_nodes = 19,min_samples_leaf = 2 )

clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
                #gini.append(tuple([i,j,k,l,metrics.accuracy_score(y3_test,y3_pred)]))
print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

Accuracy: 0.8461538461538461


In [17]:
#for deploying model the packages used are
import pickle
import requests
import json

In [18]:
pickle.dump(clf, open('model.pkl','wb'))

In [39]:
from sklearn.tree import _tree
import numpy as np
d = {0 : "no chance of heart disease",1: "chance of heart disease"}
def tree_to_code(tree, feature_names,file):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    file.write("def tree({}):\n".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "        " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            #print ("{}if {} <= {}:".format(indent, name, threshold))
            file.write("{}if {} <= {}:\n".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            #print ("{}else:  # if {} > {}".format(indent, name, threshold))
            file.write("{}else:  # if {} > {}\n".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            val = np.argmax(tree_.value[node][0])
            #print("{}return {}".format(indent,d[val]))
            file.write("{}return {}\n".format(indent,d[val]))

    recurse(0, 1)

In [24]:
tree_to_code(clf,features)

def tree(thal, thalach, slope, exang, cp, oldpeak, ca, sex):
    if thal <= 2.5:
        if oldpeak <= 1.699999988079071:
            if ca <= 0.5:
                if thalach <= 158.5:
                    return chance of heart disease
                else:  # if thalach > 158.5
                    return chance of heart disease
            else:  # if ca > 0.5
                if cp <= 0.5:
                    return no chance of heart disease
                else:  # if cp > 0.5
                    if slope <= 1.5:
                        if thal <= 1.5:
                            return no chance of heart disease
                        else:  # if thal > 1.5
                            return chance of heart disease
                    else:  # if slope > 1.5
                        return chance of heart disease
        else:  # if oldpeak > 1.699999988079071
            if cp <= 1.5:
                return no chance of heart disease
            else:  # if cp > 1.5
              

In [40]:
f = open("code1.py",'w')

In [41]:
#f.write(tree_to_code(clf,features))
tree_to_code(clf,features,f)

In [42]:
f.close()